In [1]:
import pickle
from helper_functions import create_agents, get_agent
from CP_algo import CP

## An Artificial Data Set

This dataset was generated with reference to an actual example of a Japanese local government.

- `children_dic`, `daycares_dic`, `families_dic`

In [2]:
children_dic, daycares_dic, families_dic = pickle.load(open("example_data.pkl", "rb"))

## Parameters

- `share_bool`: bool (default True)
    - whether transferable quotas are allowed or not

- `bp_num`: integer  (default 0)
    - the number of maximum blocking coalitions

- `solver_time` integer (default 1000)
    - maximum running time (seconds)

- `exclude_bool`: bool (default True)
    - for a different non-wastefulness concepts

- `search_depth`: integer (default 0)
    - the number of maximum indifferences in daycares' priorities

In [3]:
share_bool = True
bp_num = 0
solver_time = 1000
exclude_bool = False
search_depth = 0

 ## check CP
 

In [4]:
outcome_children_dic, outcome_f_CP = CP(
    children_dic,
    daycares_dic,
    families_dic,
    share_bool,
    bp_num,
    solver_time,
    exclude_bool,
    search_depth,
)

OPTIMAL
Totally matched: 497.0
solver time elapsed 0.6150012016296387


## calculate blocking coalitions

`outcome_f[f.id, p]`: for each f and for each p in range(len(f.pref))

- binary (0 or 1)
- f.pref = (1,1), (2,2), (3,3)、when the assignment is (2,2)
- `outcome_f[f.id, 0] = 0`, `outcome_f[f.id, 1] = 1`, `outcome_f[f.id, 2] = 0`

In [5]:
def check_bp(children_dic, daycares_dic, families_dic, outcome_f):
    # create CP_agents
    children, daycares, families = create_agents(
        children_dic, daycares_dic, families_dic
    )

    # update f.assignment & c.assigned_daycare
    for f in families:
        for p in range(len(f.pref)):
            if outcome_f[f.id, p] == 1:
                f.assignment = p

                for cid in f.children:
                    c = get_agent(cid, children)
                    c.assigned_daycare = c.projected_pref[p]

        if f.assignment == None:
            f.assignment = len(f.pref)

    # blocking coalition
    bp_dic = {}
    for f in families:
        bp_dic[f.id] = []

    for f in families:
        # ignore all families who are matched to their top choice
        if f.assignment != 0:
            # check every position prior to f.assignment
            for p in range(f.assignment):
                # D(f, p)
                D_fp = f.return_daycare_id_for_certain_position(p)
                bool_all = 0

                for d in D_fp:
                    # default all 0 (could accept C_fpd)
                    bool_fpd = [0 for i in range(6)]
                    # C(f,p,d)
                    C_fpd = f.return_children_for_certain_position_and_daycare(p, d)
                    for g in range(6):
                        # C(f,p,d,g)
                        C_fpdg = f.return_siblings_for_certain_position_daycare_age(
                            p, d, g, False, children, daycares
                        )
                        if len(C_fpdg) != 0:
                            w_fpd = f.return_lowest_sibling_for_certain_position_daycare_age(
                                p, d, g, False, children, daycares
                            )
                            daycare = get_agent(d, daycares)
                            better_cid = daycare.return_weak_better_children_than_child_excluding_siblings(
                                w_fpd, children, False, True, 0
                            )
                            better_number = 0
                            for x in better_cid:
                                xc = get_agent(x, children)
                                if xc.assigned_daycare == d:
                                    better_number += 1

                            if (
                                better_number + len(C_fpdg)
                                > daycare.recruiting_numbers[g]
                            ):
                                bool_fpd[g] = 1

                    if sum(bool_fpd) == 0:
                        bool_all += 1

                if bool_all == len(D_fp):
                    if p not in bp_dic[f.id]:
                        bp_dic[f.id].append(p)

    return bp_dic

In [6]:
bp_dic = check_bp(children_dic, daycares_dic, families_dic, outcome_f_CP)

bp_num = 0
for key in bp_dic.keys():
    bp_num += len(bp_dic[key])

print("number of blocking coalition is", bp_num)

number of blocking coalition is 0
